# Linear Regression - Figure Generation

This notebook generates figures for the linear regression chapter with consistent matplotlib styling.

In [ ]:
# Setup and styling
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import proj3d

# Apply our custom matplotlib style
plt.style.use('notes-base.mplstyle')

In [ ]:
# 3D Loss Surface Plot for Linear Regression
# --- data ---
rng = np.random.default_rng(3)
x = np.linspace(0, 2, 20)
y = 0.9 + 0.35 * x + rng.normal(0, 0.15, len(x))

X = np.c_[np.ones_like(x), x]
phi0_hat, phi1_hat = np.linalg.lstsq(X, y, rcond=None)[0]

phi0 = np.linspace(0, 2.2, 200)
phi1 = np.linspace(-1.0, 1.0, 200)
P0, P1 = np.meshgrid(phi0, phi1)
res = y[:, None, None] - (P0[None, ...] + P1[None, ...] * x[:, None, None])
L = (res**2).mean(axis=0)

cand = np.array([[0.2, 0.9],
                 [1.8, 0.8],
                 [phi0_hat, phi1_hat]])
cand_losses = np.array([((y - (c0 + c1*x))**2).mean() for c0, c1 in cand])

# --- plot ---
fig = plt.figure(figsize=(11, 5.2))

# (a) 3D loss surface
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
surf = ax1.plot_surface(P0, P1, L, rstride=4, cstride=4, cmap='magma',
                        linewidth=0, antialiased=True, alpha=0.75)
surf.set_zsort('min')          # draw far facets first
ax1.set_xlabel('Intercept')
ax1.set_ylabel('Slope')
ax1.set_zlabel('Loss')
ax1.set_title('Loss (Surface Plot)')
ax1.view_init(elev=28, azim=-55)

# (b) heatmap + isocontours
ax2 = fig.add_subplot(1, 2, 2)
im = ax2.contourf(P0, P1, L, levels=60, cmap='magma')
ax2.contour(P0, P1, L, levels=12, colors='white', linewidths=0.6, alpha=0.7)
ax2.scatter(cand[:,0], cand[:,1], s=60,
            c=['yellow','tab:cyan','limegreen'],
            edgecolors='k', linewidths=0.7)
ax2.set_xlabel('Intercept')
ax2.set_ylabel('Slope')
ax2.set_title('Loss (Contour Plot)')
cbar = fig.colorbar(im, ax=ax2, fraction=0.046, pad=0.04)
cbar.set_label('Loss')

plt.subplots_adjust(wspace=0.4)
fig.canvas.draw()  # finalize layout/projection before overlay

# -------- overlay markers on 3D axes (robust; no tight-bbox errors) --------
# project 3D points to 2D in ax1's data coordinates
x2, y2, _ = proj3d.proj_transform(cand[:,0], cand[:,1], cand_losses, ax1.get_proj())

# create a plain Axes that exactly overlaps ax1 (no locator; excluded from layout)
bbox = ax1.get_position()
ax_overlay = fig.add_axes(bbox, frameon=False)
ax_overlay.set_axis_off()
ax_overlay.set_in_layout(False)

# convert display pixels -> overlay's axes fraction and draw there
xy_disp = ax1.transData.transform(np.c_[x2, y2])
xy_axes = ax_overlay.transAxes.inverted().transform(xy_disp)
ax_overlay.scatter(xy_axes[:,0], xy_axes[:,1], s=120,
                   transform=ax_overlay.transAxes,
                   c=['yellow','tab:cyan','limegreen'],
                   edgecolors='k', linewidths=1.2, clip_on=True)

# Save for LaTeX
plt.savefig("../tex/images/linear_reg_loss_surface.pdf", transparent=True)
plt.show()